# Importing Data

In [1]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",
                       engine="python",
                       sep=" ::: ",
                       names=["id", "movie", "genre", "summary"])

test_df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt",
                      engine="python",
                      sep=" ::: ",
                      names=["id", "movie", "genre", "summary"])

In [2]:
# Viewing training data
train_df.head()

,id,movie,genre,summary
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


In [3]:
# Viewing test data
test_df.head()

,id,movie,genre,summary
0,1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty chi..."
2,3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family o...
3,4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),drama,Before he was known internationally as a marti...


# Data Manipulation

## Shuffling the data

In [4]:
train_shuffled = train_df.sample(frac=1)

## Splitting the data

In [5]:
# Split the data using train_test_split from sklearn
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_shuffled["summary"],
                                                 train_shuffled["genre"],
                                                 test_size=0.1)
X_test, y_test = test_df["summary"], test_df["genre"]

## One-Hot Encoding (Labels)

In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
ohe.fit(train_shuffled["genre"].to_numpy().reshape(-1,1)) # Fit the encoder to genre of training data

train_ohe = ohe.transform(y_train.to_numpy().reshape(-1, 1))
val_ohe = ohe.transform(y_val.to_numpy().reshape(-1,1))
test_ohe = ohe.transform(y_test.to_numpy().reshape(-1,1))

## List (Summary)

In [7]:
train_sentences = X_train.tolist()
val_sentences = X_val.tolist()
test_sentences = X_test.tolist()

# Universal Serial Encoder (Embedding Layer)

In [8]:
import tensorflow_hub as hub
embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
                                        name="universal_sentence_encoder")

# Creating Dataset

In [9]:
from tensorflow.data import Dataset as tfd
import tensorflow as tf

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = tfd.from_tensor_slices((X_train, train_ohe)).batch(32).prefetch(AUTOTUNE)
val_dataset = tfd.from_tensor_slices((X_val, val_ohe)).batch(32).prefetch(AUTOTUNE)
test_dataset = tfd.from_tensor_slices((X_test, test_ohe)).batch(32).prefetch(AUTOTUNE)

train_dataset, val_dataset, test_dataset

(<PrefetchDataset shapes: ((None,), (None, 27)), types: (tf.string, tf.float64)>,
 <PrefetchDataset shapes: ((None,), (None, 27)), types: (tf.string, tf.float64)>,
 <PrefetchDataset shapes: ((None,), (None, 27)), types: (tf.string, tf.float64)>)

# Model

In [10]:
classes = len(train_shuffled["genre"].value_counts())

In [11]:
# Build the model
from tensorflow.keras import layers

inputs = layers.Input(shape=[], dtype="string")
x = embedding_layer(inputs)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

In [12]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
universal_sentence_encoder ( (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 27)                13851     
Total params: 257,074,331
Trainable params: 276,507
Non-trainable params: 256,797,824
_________________________________________________________________


In [14]:
history = model.fit(train_dataset,
                  steps_per_epoch=int(0.1*len(train_dataset)),
                  epochs=5,
                  validation_data=val_dataset,
                  validation_steps=int(0.1*len(val_dataset)))

Epoch 1/5
152/152 [==============================] - 7s 28ms/step - loss: 2.3949 - accuracy: 0.3906 - val_loss: 1.7015 - val_accuracy: 0.5349
Epoch 2/5
152/152 [==============================] - 4s 24ms/step - loss: 1.6297 - accuracy: 0.5328 - val_loss: 1.5085 - val_accuracy: 0.5790
Epoch 3/5
152/152 [==============================] - 4s 30ms/step - loss: 1.5102 - accuracy: 0.5522 - val_loss: 1.3929 - val_accuracy: 0.5809
Epoch 4/5
152/152 [==============================] - 4s 24ms/step - loss: 1.4497 - accuracy: 0.5546 - val_loss: 1.3214 - val_accuracy: 0.6029
Epoch 5/5
152/152 [==============================] - 4s 23ms/step - loss: 1.3245 - accuracy: 0.5943 - val_loss: 1.2771 - val_accuracy: 0.6158


In [15]:
model.evaluate(test_dataset)

1694/1694 [==============================] - 37s 22ms/step - loss: 1.3660 - accuracy: 0.5783


[1.365978479385376, 0.5782656669616699]

# Conclusion

Accuracy: 57.88%

Cleaning the data strangely gives lesser accuracy.